In [4]:
! pip list

Package                     Version
--------------------------- -----------
aioconsole                  0.8.1
aiofiles                    24.1.0
aiohappyeyeballs            2.4.4
aiohttp                     3.11.10
aiosignal                   1.3.1
annotated-types             0.7.0
anyio                       4.7.0
appnope                     0.1.4
asttokens                   3.0.0
asyncio-atexit              1.0.1
attrs                       24.2.0
autogen-agentchat           0.4.0.dev11
autogen-core                0.4.0.dev11
autogen-ext                 0.4.0.dev11
azure-ai-ml                 1.22.4
azure-ai-projects           1.0.0b2
azure-common                1.1.28
azure-core                  1.32.0
azure-identity              1.19.0
azure-mgmt-core             1.5.0
azure-storage-blob          12.24.0
azure-storage-file-datalake 12.18.0
azure-storage-file-share    12.20.0
cachetools                  5.5.0
certifi                     2024.8.30
cffi                        1.17.1
c

In [1]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
import os

In [2]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='Your Azure AI Foundation Connection String',
)

In [3]:

with project_client:

    code_interpreter = CodeInterpreterTool()
    
    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="my-agent",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
        # tool_resources=code_interpreter.resources,
    )

    thread = project_client.agents.create_thread()

    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="""
        
        You are my Python programming assistant. Generate code and execute it according to the following requirements

        1. Save "this is blog" to blog-{YYMMDDHHMMSS}.md

        2. give me the download this file link
            """,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # # delete the original file from the agent to free up space (note: this does not delete your version of the file)
    # project_client.agents.delete_file(file.id)
    # print("Deleted file")

    # print the messages from the agent
    messages = project_client.agents.get_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_sender("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

    # print(f"File: {messages.file_path_annotations}")


    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")


    

Run finished with status: RunStatus.COMPLETED
Messages: <azure.ai.projects.models._patch.ThreadMessages object at 0x108692bd0>
Last Message: The file has been saved as `blog-241231132844.md`. You can download it using the link below:

[Download blog-241231132844.md](sandbox:/mnt/data/blog-241231132844.md)
